<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC213.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ABC213](https://atcoder.jp/contests/abc213)

## [A - Bitwise Exxxclusive Or](https://atcoder.jp/contests/abc213/tasks/abc213_a)

Xorは, ある桁の10-1のbitが同じだと, その桁は0となるので,  $A xor A = 0$ となる. 

よって, $A xor C = B$ の左から$Aをxor$して 
$$
A xor A xor C = A xor B \\
0 xor C = A xor B \\
C = A xor B
$$
となる.

In [1]:
A, B = map(int, input().split())

10 12


In [2]:
print(A ^ B)

6


## [B - Booby Prize](https://atcoder.jp/contests/abc213/tasks/abc213_b)

In [ ]:
N = int(input())
A = list(map(int, input().split()))

5
3 1 4 15 9


In [ ]:
B = []
for i in range(N):
  B.append((i, A[i]))
B.sort(key=lambda x:x[1])

In [ ]:
print(B[-2][0] + 1)

5


## [C - Reorder Cards](https://atcoder.jp/contests/abc213/tasks/abc213_c)

座標圧縮をする問題.


### 2分探索を使う方法
1. 縦軸の座標A （使われるすべての座標一覧）
2. Aの要素んぼ重複を消してソートした配列 A'（使われている座標の昇順, 重複なし）
3. Aの各要素はdAの何番目かを探索（二分探索bisectを使う）

これを横の座標Bにも同様に適応

In [1]:
from bisect import bisect_left
H, W, N = map(int, input().split())
A, B = [], []
for _ in range(N):
  a, b = map(int, input().split())
  A.append(a)
  B.append(b)

dA = sorted(list(set(A)))
dB = sorted(list(set(B)))

for a,b in zip(A,B):
  a_ind = bisect_left(dA, a)
  b_ind = bisect_left(dB, b)
  print(a_ind+1, b_ind+1)

4 5 2


### 辞書を使う方法
1. 縦軸の座標A （使われるすべての座標一覧）
2. Aの要素んぼ重複を消してソートした配列 A'（使われている座標の昇順, 重複なし）
3. A'の要素の値をkey, A'において何番目かをvalueにした辞書A''
4. 各Aの要素のA''における値を取得

これを横の座標Bにも同様に適応

In [15]:
H, W, N = map(int, input().split())
A, B = [], []
for _ in range(N):
  a,b = map(int, input().split())
  A.append(a)
  B.append(b)

dA = sorted(list(set(A)))
dB = sorted(list(set(B)))

ddA = {x:i+1 for i, x in enumerate(dA)}
ddB = {y:j+1 for j, y in enumerate(dB)}

for i in range(N):
  print(ddA[A[i]], ddB[B[i]])

4 5 2
3 2
2 5
2 1
1 2


## [D - Takahashi Tour](https://atcoder.jp/contests/abc213/tasks/abc213_d)

深さ優先探索を行えば良い. 「今どの頂点にいるか」を表示しないといけないが, それをdfsのどのタイミングで行えば良いのかが難しい.

再帰的にdfsを行うが, 
1. その頂点を訪れた瞬間
2. その頂点での再帰dfsが終わった瞬間

に頂点を追加すると良い.

Gの隣接リストをソートしておくのを忘れずに.

In [21]:
import sys
sys.setrecursionlimit(10**9) 
N = int(input())
G = [[] for _ in range(N+1)]
for _ in range(N-1):
  A, B = map(int, input().split())
  G[A].append(B)
  G[B].append(A)
for i in G:
  i.sort()

4
1 2
4 2
3 1


In [30]:
ans = []
def dfs(bef, now):
  ans.append(now)
  for i in G[now]:
    if i != bef:
      dfs(now, i)
      # dfs(now,i)が終わった後は, nowにいる.
      # そして, nowに隣接している次の頂点に移動することになる
      ans.append(now)

In [31]:
dfs(-1, 1)
print(*ans)

1 2 4 2 1 3 1


## [E - Stronger Takahashi](https://atcoder.jp/contests/abc213/tasks/abc213_e)

隣接する2×2の区間を破壊すると, 今いる箇所からマンハッタン距離が3以内のところにコスト1で移動できることになる.

壁のない箇所にはコスト0で移動できる.

こう考えると, 辺のコストが0 or 1の最短経路問題と考えることができる. 01-BFSを使って解く.

01-BFSの説明 : https://betrue12.hateblo.jp/entry/2018/12/08/000020

In [ ]:
from collections import deque
from itertools import product

dh = [-1, 1, 0, 0]
dw = [0, 0, -1, 1]

# mov1 : 隣接頂点への移動
mov1 = tuple((h, w) for h, w in zip(dh, dw))

# mov2 : 2*2の壁を破壊しての移動. マンハッタン距離が1以上3以下の箇所に動ける
mov2 = tuple((h,w) for h,w in product(range(-2, 3), repeat=2) if 1 <= abs(h) + abs(w) <= 3)

In [ ]:
H, W = map(int, input().split())
grid = []
for _ in range(H):
  temp = input()
  grid.append(temp)

In [ ]:
# 0-1BFS
dist = [[10**10] * W for _ in range(H)]
dq = deque()

# (その頂点までの最短cost, 座標h, 座標w)
dq.append((0, 0, 0))
while dq:
  c, h, w = dq.popleft()
  # mov1の移動は壁を通れないのでコストの変化はない
  for dh, dw in mov1:
    # 次の座標
    nh, nw = h + dh, w + dw
    # gridからはみ出ないかのチェック
    if 0 <= nh < H and 0 <= nw < W:
      if grid[nh][nw] == "." and c < dist[nh][nw]:
        dist[nh][nw] = c
        dq.appendleft((c, nh, nw))

  # mov2の移動はコストの変化あり
  for dh, dw in mov2:
    nh, nw = h + dh, w + dw
    if 0 <= nh < H and 0 <= nw < W:
      # マスに壁があるかどうかの判定はいらない
      if c + 1 < dist[nh][nw]:
        dist[nh][nw] = c + 1
        dq.append((c + 1, nh, nw))

print(dist[-1][-1])